In [1]:
# Somehow in mac, it cant find local packages
# Run this to solve no local modules found in mac
import os
import sys

# Get the project root directory (assuming your notebook is in a subdirectory)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


In [13]:
# Python lib
import asyncio
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from typing import Dict, Any, Tuple, List
import venv
import os

# Autogen-0.4
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.agents._code_executor_agent import CodeExecutorAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai.config import OpenAIClientConfigurationConfigModel
from autogen_core.model_context import BufferedChatCompletionContext

# Local
from utils import GetDatasetProfile

In [3]:
root = '../sheets/mysql/'
files: list[str] = os.listdir(path=root)
tasks = [GetDatasetProfile(root, file_name = file, output_format='json') for file in files]
results = await asyncio.gather(*tasks)

Processing 13 columns (customer.csv)                                   ██████████████████████████████|  13/13  
Processing 8 columns (employee.csv)                                    ██████████████████████████████|   8/8   
Processing 9 columns (office.csv)                                      ██████████████████████████████|   9/9   
Processing 5 columns (orderdetails.csv)                                ██████████████████████████████|   5/5   
Processing 7 columns (orders.csv)                                      ██████████████████████████████|   7/7   
Processing 4 columns (payments.csv)                                    ██████████████████████████████|   4/4   
Processing 4 columns (productlines.csv)                                ██████████████████████████████|   4/4   
Processing 9 columns (products.csv)                                    ██████████████████████████████|   9/9   


In [4]:
result_dict = zip(files, results)
results = list(result_dict)

In [21]:
# Only edit here AND filepath under if __name__ == "__main__":
reasoning_model = "qwen2.5:32b-instruct-q8_0-32768"
coding_model = "qwen2.5-coder:32b-instruct-q8_0"

# Common config
llm_base_url = "http://34.204.63.234:11434/v1"
api_key = "none"
model_info =  {
        "vision": False,
        "function_calling": False,
        "json_output": True,
        "family": "Qwen2.5"
    }

#######################################################################
#   !!! DONT EDIT BELOW EXCEPT FOR if __name__ == "__main__":   !!!   #
#######################################################################


# Create reasoning config
# Need to be more creative
data_dict_summarizer_config = OpenAIClientConfigurationConfigModel(
    frequency_penalty=0.2, 
    logit_bias=None, 
    max_tokens=128000, 
    n=None, 
    presence_penalty=0.5, 
    response_format=None, 
    seed=42, 
    stop=None, 
    temperature=0.8, 
    top_p=0.95, 
    user=None,
    model=reasoning_model,
    api_key='none', 
    timeout=None, 
    max_retries=None, 
    model_info=model_info, 
    organization=None, 
    base_url='http://34.204.63.234:11434/v1'
    )

# Just generate the data dict based on obervered facts
data_dict_generator_config = OpenAIClientConfigurationConfigModel(
    frequency_penalty=0.4, 
    logit_bias=None, 
    max_tokens=2048, 
    n=None, 
    presence_penalty=0.3, 
    response_format={"type": "json_object"},
    seed=42, 
    stop=None, 
    temperature=0.2, 
    top_p=0.7, 
    user=None,
    model=reasoning_model,
    api_key='none', 
    timeout=None, 
    max_retries=None, 
    model_info=model_info, 
    organization=None, 
    base_url='http://34.204.63.234:11434/v1'
    )



In [28]:
data_dict_summarizer_client = OpenAIChatCompletionClient(model=reasoning_model, model_info=model_info)._from_config(data_dict_summarizer_config)
data_dict_generator_client = OpenAIChatCompletionClient(model=reasoning_model, model_info=model_info)._from_config(data_dict_generator_config)

In [29]:
results[0][0]

'customer.csv'

In [30]:
async def fix_file_name(filename):
    # Turn file name into Python identifier
    # Since Python identifier can't contain '.', we replace it with _
    last_5_chars = list(filename)[-5:]
    replace_dot = [char.replace('.', '_') for char in last_5_chars]
    all_chars_before_last_5 = list(filename)[:-5]
    all_chars_before_last_5.extend(replace_dot)
    return ''.join(all_chars_before_last_5)

In [31]:
await fix_file_name(results[0][0])

'customer_csv'

In [34]:

async def initialize_individual_chat(filename: str, metadata):
    
    file_name_fixed = await fix_file_name(filename)
    
    # Initialize agents dynamically
    file_handler = AssistantAgent(
        name=f"File_handler_{file_name_fixed}",
        description=f"A file handling agent specific for the file {filename}.",
        model_client=data_dict_generator_client,
        system_message=f"""<purpose>You are a file handling agent for the file {filename}. You will populate a data dictionary for this particular dataset.</purpose>
        
<instructions>Given the metadata of the file, suggest possible meanings and explain abbreviations of columns in the file. Suggest common column names that would be analyzed together. MUST follow output_format.</instructions>

<output_format>
Data Dictionary for {filename}:

---

Column: <column_name>
Possible meaning: <meaning>
Description: <description>
Relationships with other columns: <relationships>
Sample value: <1 sample value>

---
</output_example>

<rules>
OMIT performing any other actions other than those specified.
OMIT speaking more than necessary.
Must follow output_format.
</rules>""",
    )
    
    response = await file_handler.on_messages(
        [TextMessage(content=f"You are a file handling agent for the file {filename}. The file metadata is as follow: {str(metadata)}. Suggest possible meanings and abbreviations of columns in the file.", source="user")], None
    )
    return response



In [35]:
# Create a list of tasks for all agents
tasks = [initialize_individual_chat(filename=results[index][0], metadata=results[index][1]) for index in range(len(results))]
    
# Execute all agent chat tasks concurrently
responses = await asyncio.gather(*tasks)

# Example usage
print(responses)
print(f"\nResponses received: {len(responses)}")

[Response(chat_message=TextMessage(source='File_handler_customer_csv', models_usage=RequestUsage(prompt_tokens=1099, completion_tokens=1418), content='{\n"Data Dictionary for customer.csv": [\n    {\n        "Column": "customerNumber",\n        "Possible meaning": "Unique identifier for each customer.",\n        "Description": "An integer that uniquely identifies a specific customer record within the dataset.",\n        "Relationships with other columns": "This column is likely to be used as a foreign key in related tables such as orders or payments.",\n        "Sample value": 103\n    },\n    {\n        "Column": "customerName",\n        "Possible meaning": "The name of the customer\'s company or individual.",\n        "Description": "A string that represents the full name of the customer, which could be a business entity or an individual person.",\n        "Relationships with other columns": "This column is often used in conjunction with contactLastName and contactFirstName to identi

In [38]:
print(responses[5].chat_message.content)

{
"Data Dictionary for payments.csv": [
    {
      "Column": "customerNumber",
      "Possible meaning": "Unique identifier for each customer.",
      "Description": "An integer value that uniquely identifies a specific customer within the dataset.",
      "Relationships with other columns": "This column could be used to link payment records back to individual customers, possibly correlating with data in another table or file containing detailed customer information.",
      "Sample value": 103
    },
    {
      "Column": "checkNumber",
      "Possible meaning": "Unique identifier for each check/payment transaction.",
      "Description": "A string that uniquely identifies a payment made by the customer, likely used to track individual transactions or checks.",
      "Relationships with other columns": "This column could be related to 'paymentDate' and 'amount', as it represents an instance of a specific amount being paid on a certain date.",
      "Sample value": "HQ336336"
    },
 

In [37]:
import json
pydict = json.loads(responses[5].chat_message.content)
pydict

{'Data Dictionary for payments.csv': [{'Column': 'customerNumber',
   'Possible meaning': 'Unique identifier for each customer.',
   'Description': 'An integer value that uniquely identifies a specific customer within the dataset.',
   'Relationships with other columns': 'This column could be used to link payment records back to individual customers, possibly correlating with data in another table or file containing detailed customer information.',
   'Sample value': 103},
  {'Column': 'checkNumber',
   'Possible meaning': 'Unique identifier for each check/payment transaction.',
   'Description': 'A string that uniquely identifies a payment made by the customer, likely used to track individual transactions or checks.',
   'Relationships with other columns': "This column could be related to 'paymentDate' and 'amount', as it represents an instance of a specific amount being paid on a certain date.",
   'Sample value': 'HQ336336'},
  {'Column': 'paymentDate',
   'Possible meaning': 'The d

In [39]:
# Extract class:: TextMessage from class:: Response
# Since agents only accept [TextMessage] as input prompt
responses_TextMessage = [json.loads(responses[index].chat_message.content) for index in range(len(responses))]
responses_TextMessage

[{'Data Dictionary for customer.csv': [{'Column': 'customerNumber',
    'Possible meaning': 'Unique identifier for each customer.',
    'Description': 'An integer that uniquely identifies a specific customer record within the dataset.',
    'Relationships with other columns': 'This column is likely to be used as a foreign key in related tables such as orders or payments.',
    'Sample value': 103},
   {'Column': 'customerName',
    'Possible meaning': "The name of the customer's company or individual.",
    'Description': 'A string that represents the full name of the customer, which could be a business entity or an individual person.',
    'Relationships with other columns': 'This column is often used in conjunction with contactLastName and contactFirstName to identify specific contacts within a customer record.',
    'Sample value': 'Atelier graphique'},
   {'Column': 'contactLastName',
    'Possible meaning': 'The last name of the primary contact person for this customer.',
    'Des

In [40]:
prompt = """<Data_Dictionary_Analytics_Suggester>
    
    <purpose>
        Analyze provided data dictionaries comprehensively to identify column relationships and suggest a wide range of potential analytics use cases that leverage these relationships to generate meaningful and actionable business insights.
    </purpose>
    
    <instructions>
        1. **Review** all provided data dictionaries thoroughly, ensuring no dataset or column is overlooked.
        2. **Examine** each dataset's columns, their meanings, descriptions, and existing relationships in detail.
        3. **Identify** all meaningful relationships and combinations of columns within the same dataset and across different datasets that can be leveraged for insightful analysis.
        4. **Ensure** that every dataset and its relevant columns are considered to maximize the utilization of available data.
        5. **Brainstorm** a diverse range of potential analytics use cases that utilize these column relationships to address various business objectives such as improving sales, optimizing inventory, understanding customer behavior, enhancing operational efficiency, etc.
        6. **For each suggested use case**, provide a detailed explanation of how specific column combinations and their relationships contribute to the insights.
        7. **Ensure** that all suggested use cases are actionable, relevant, and based solely on the information provided in the data dictionaries without incorporating any external knowledge or assumptions.
        8. **Summarize** key relationships between datasets that underpin the proposed analytics use cases, ensuring that cross-dataset relationships are highlighted and utilized.
    </instructions>
    
    <output_format>
        For each identified analytics use case, provide the following structured information:
        
        ```
        ### Use Case <Number>
        - **Use Case ID:** UC<UniqueNumber>
        - **Title:** <Descriptive Title of the Use Case>
        - **Description:** <Detailed explanation of the analytics use case, including the business objective it addresses and the insights it aims to generate.>
        - **Data Sources:**
            - <Dataset1.csv>
            - <Dataset2.csv>
            - <!-- Add additional data sources as needed -->
        - **Columns Utilized:**
            - <Dataset1.csv>.<ColumnName>
            - <Dataset2.csv>.<ColumnName>
            - <!-- List all relevant columns from the respective datasets -->
        - **Relationships Leveraged:**
            - <Description of how the columns are related (e.g., primary key, foreign key, common attributes) and how these relationships are utilized in the analysis.>
        
        ---
        ```
        
        **Example:**
        
        ```
        ### Use Case 1
        - **Use Case ID:** UC001
        - **Title:** Inventory Optimization Based on Product Categories
        - **Description:** Analyze the `quantityInStock` in relation to `productLine` and `buyPrice` to identify overstocked or understocked categories, enabling better inventory management and cost optimization.
        - **Data Sources:**
            - products.csv
            - productlines.csv
        - **Columns Utilized:**
            - products.csv.quantityInStock
            - products.csv.buyPrice
            - products.csv.productLine
            - productlines.csv.productLine
        - **Relationships Leveraged:**
            - The `productLine` column in `products.csv` is related to the `productLine` column in `productlines.csv`, allowing categorization of inventory levels by product category.
        
        ---
        ```
    </output_format>
    
    <rules>
        1. **Exact Naming:** Use the exact table and column names as specified in the provided data dictionaries.
        2. **Relevance:** Only suggest analytics use cases that are directly supported by the relationships and data available in the data dictionaries.
        3. **Actionable Use Cases:** Ensure that each use case is actionable and aligned with common business objectives such as improving sales, optimizing inventory, understanding customer behavior, enhancing operational efficiency, etc.
        4. **Comprehensive Utilization:** Actively utilize as many datasets and columns as possible, ensuring that no relevant data is left unexplored or unused in the suggested use cases.
        5. **Clarity:** Provide clear and concise descriptions for each use case, avoiding ambiguity and ensuring that the purpose and methodology are easily understandable.
        6. **Structured Output:** Adhere strictly to the specified output format to maintain consistency and ease of interpretation across all suggested use cases.
        7. **No Assumptions:** Base all suggestions solely on the provided data dictionaries without introducing external information or making assumptions beyond the given data.
        8. **Unique Identification:** Assign a unique ID to each use case to facilitate easy reference and tracking.
        9. **Avoid Redundancy:** Ensure that each use case is unique and does not duplicate the purpose or methodology of another use case.
        10. **Cross-Dataset Relationships:** Actively seek and leverage relationships across different datasets to create comprehensive and insightful use cases that span multiple areas of the business.
    </rules>
    
</Data_Dictionary_Analytics_Suggester>"""



In [41]:
# Construct a list of TextMessage as prompt
# Combine data dict ingestion agent prompt with all previous responses
initial_task = [TextMessage(content=f"Review the data dictionary generated by the file handling agents.",source="user")]
responses_TextMessage.extend(initial_task)

In [42]:
responses_TextMessage

[{'Data Dictionary for customer.csv': [{'Column': 'customerNumber',
    'Possible meaning': 'Unique identifier for each customer.',
    'Description': 'An integer that uniquely identifies a specific customer record within the dataset.',
    'Relationships with other columns': 'This column is likely to be used as a foreign key in related tables such as orders or payments.',
    'Sample value': 103},
   {'Column': 'customerName',
    'Possible meaning': "The name of the customer's company or individual.",
    'Description': 'A string that represents the full name of the customer, which could be a business entity or an individual person.',
    'Relationships with other columns': 'This column is often used in conjunction with contactLastName and contactFirstName to identify specific contacts within a customer record.',
    'Sample value': 'Atelier graphique'},
   {'Column': 'contactLastName',
    'Possible meaning': 'The last name of the primary contact person for this customer.',
    'Des

In [43]:
# Initialize data dict ingestion agent
final_agent = AssistantAgent(name="Data_Dictionary_Analytics_Suggester",
                             model_client=data_dict_summarizer_client,
                             system_message=prompt
)

response = await final_agent.on_messages(
        [TextMessage(content=f"{responses_TextMessage}\n\nReview the above data dictionary.\n\n{responses_TextMessage}",source="user")], None
    )
response

Response(chat_message=TextMessage(source='Data_Dictionary_Analytics_Suggester', models_usage=RequestUsage(prompt_tokens=12734, completion_tokens=1782), content="### Use Case 1\n- **Use Case ID:** UC001\n- **Title:** Inventory Optimization Based on Product Categories\n- **Description:** Analyze `quantityInStock` in relation to `productLine` and `buyPrice` to identify overstocked or understocked categories, enabling better inventory management and cost optimization.\n- **Data Sources:**\n    - products.csv\n    - productlines.csv\n- **Columns Utilized:**\n    - products.csv.quantityInStock\n    - products.csv.buyPrice\n    - products.csv.productLine\n    - productlines.csv.productLine\n- **Relationships Leveraged:**\n    - The `productLine` column in `products.csv` is related to the `productLine` column in `productlines.csv`, allowing categorization of inventory levels by product category.\n    \n---\n### Use Case 2\n- **Use Case ID:** UC002\n- **Title:** Sales Performance Analysis by Pr

In [44]:
print(response.chat_message.content)

### Use Case 1
- **Use Case ID:** UC001
- **Title:** Inventory Optimization Based on Product Categories
- **Description:** Analyze `quantityInStock` in relation to `productLine` and `buyPrice` to identify overstocked or understocked categories, enabling better inventory management and cost optimization.
- **Data Sources:**
    - products.csv
    - productlines.csv
- **Columns Utilized:**
    - products.csv.quantityInStock
    - products.csv.buyPrice
    - products.csv.productLine
    - productlines.csv.productLine
- **Relationships Leveraged:**
    - The `productLine` column in `products.csv` is related to the `productLine` column in `productlines.csv`, allowing categorization of inventory levels by product category.
    
---
### Use Case 2
- **Use Case ID:** UC002
- **Title:** Sales Performance Analysis by Product Line and Region
- **Description:** Analyze sales performance using `quantityOrdered`, `priceEach`, and the corresponding `productLine` to identify high-demand categories in 